In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from random import sample
from collections import Counter
import string

In [2]:
def readTxt(fileName):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3\Datasets' + fileName
    df = pd.read_csv(fullFileName, encoding='utf-8', header = None,
                 sep='\t')
    
    return df

In [3]:
def readCsv(fileName):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3' + fileName
    df = pd.read_csv(fullFileName, encoding='utf-8', header = None,
                 sep=',')
    
    return df

In [59]:
def toCsvDf(fileName, df):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3\Datasets' + fileName
    df.to_csv(fullFileName, header = False, sep='\t', index = False)

In [58]:
def toCsvNp(fileName, npArray):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3\Datasets' + fileName
    np.savetxt(fullFileName, npArray, delimiter=',')

In [107]:
def createDictionary(trainingSet):
    allWords = list()
    cnt = Counter()
    
    if trainingSet=='IMDB':
        trainingDF = readTxt('\IMDB-train.txt')
    elif trainingSet=='yelp':
        trainingDF = readTxt('\yelp-train.txt')
        
    translator = str.maketrans('','', string.punctuation)
    
    for i in range(0, len(trainingDF)):
        allWords.extend(trainingDF.iloc[i,0].translate(translator).lower().split(" "))
    
    for word in allWords:
        cnt[word] +=1

    
    
    dictionaryWords = list(zip(*cnt.most_common(10000)))[0]
    
    dictionary = {}
    for index, key in enumerate(dictionaryWords):
        dictionary[key] = index
#     print(dictionary)
    newArray = np.asarray(cnt.most_common(10000))
    withIndexVocabArray = np.insert(newArray, 1, range(0,10000),1)
    
    if trainingSet=='IMDB':
        toCsvDf('\IMDB-vocab.txt', pd.DataFrame(withIndexVocabArray))
    elif trainingSet=='yelp':
        toCsvDf('\yelp-vocab.txt', pd.DataFrame(withIndexVocabArray))
        
    return dictionary

In [76]:
def convertWordsToVector(trainingRow, dictionary):
    translator = str.maketrans('','', string.punctuation)
    returnRow = trainingRow.translate(translator).lower().split(" ")
    vector = np.zeros(10000, dtype=np.int8)
    
    for word in returnRow:
        if word in dictionary:
            vector[dictionary[word]] = np.int8(1)
            
    return vector

In [110]:
# TODO: CHANGE DICTIONARY CREATION FROM ONLY IMDB TRAINING TO YELP TRAINING AS WELL
def createBagOfWordsMatrix(df, dictionary):
#     if trainingSet=='IMDB':
#         trainingDF = readTxt('\IMDB-train.txt')
#     elif trainingSet=='yelp':
#         trainingDF = readTxt('\yelp-train.txt')

    translator = str.maketrans('','', string.punctuation)
    
    distancesArray = np.zeros((len(df), len(dictionary)), dtype=np.int8)
        
    for i in range(0, len(df)):
        vector = convertWordsToVector(df.iloc[i,0], dictionary)
        distancesArray[i] = vector

    return distancesArray
# trainingDF = readTxt('\IMDB-train.txt')
# npArray = createBagOfWordsMatrix('IMDB')
# print(npArray)
# dfArray = pd.DataFrame(npArray)
# dfArray.to_csv(r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3\Datasets\TESTING2.csv', header = False, sep=',', index = False)

In [112]:
def baselineClassifier(dataset):
    if dataset == 'yelp':
        dictionary = createDictionary('yelp')
        trainingDF = readTxt('\yelp-train.txt')
        validationDF = readTxt('\yelp-valid.txt')
    elif dataset == 'IMDB':
        dictionary = createDictionary('IMDB')
        trainingDF = readTxt('\yelp-train.txt')
        validationDF = readTxt('\yelp-valid.txt')
    from sklearn.dummy import DummyClassifier
    from numpy.random import random_integers
    randomClassifier = DummyClassifier(strategy='uniform')
    randomClassifier.fit(createBagOfWordsMatrix(trainingDF, dictionary), trainingDF[1])
    predictionsArrayRandom = randomClassifier.score(createBagOfWordsMatrix(validationDF, dictionary), validationDF[1])
    print('Random Classifier Score', predictionsArrayRandom)
    majorityClassifier = DummyClassifier(strategy='most_frequent')
    majorityClassifier.fit(createBagOfWordsMatrix(trainingDF, dictionary), trainingDF[1])
    predictionsArrayMostFreq = majorityClassifier.score(createBagOfWordsMatrix(validationDF, dictionary), validationDF[1])
    print('Majority Classifier Score', predictionsArrayMostFreq)
    
baselineClassifier()

Random Classifier Score 0.217
Majority Classifier Score 0.356
